<a href="https://colab.research.google.com/github/rashwin88/gan_clip_notebooks/blob/main/GANClipCodingSession1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First we will have to import all the necessary libraries that will help build GANs.
1. `pdb` is used for debugging.
2. `DataLoader1 is used to create iterables out of training data. Look further in the notebook for how iterables work.
3. `make_grid` is used to draw or show the images that the Generator is producing.
4. `tqdm` for beautiful progress bars and such.

We also set up a standard logger so that we can see what is going on an debug as necessary.

####TODO: Need to write something on DataLoader

In [6]:
import torch
import pdb # For debugging
from torch.utils.data import DataLoader
from torch import nn
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
# Set up some basic logging configurations
import sys
import os
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)],
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.info("Setting up logging for the project")

INFO:__main__:Setting up logging for the project


As we train, we will want to show some grid of generated images vs some real images. So we will develop some convenience functions for the same. This is actually pretty interesting because it involves some tensor operations that can seem confusing at first. The idea is to display a 4x4 grid of images (which are images output by the network or are images that are inputs from the DataLoader).

In [8]:
from typing import Tuple
def show(tensor, channels: int = 1, size: Tuple = (28,28), num: int = 16):
  """
  Displays a a set of images in a grid.

  Args:
    tensor: The tensor containing the images
    channels: The number of channels in each image.
    size: The number of width and height pixels (in that order in the image)
    num: The number of images to be displayed.
  """
  logger.debug("Initializing image display")
  logger.info("Detaching tensor from GPU and attaching it to CPU and resizing")
  data = tensor.detach().gpu().view(-1, channels, *size) # Batch Size X Channels X Width X Height
  logger.debug("Creating grid")
  grid = make_grid(data[:num], nrows=4).permute(1,2,0) # Channels X Widthj X Height -> Width X Height X Channels
  plt.imshow(grid)
  plt.show()